In [157]:
import pandas as pd
import ssl 
import numpy as np
import os
from pathlib import Path

ssl._create_default_https_context = ssl._create_unverified_context
#p_id='253802'

Table: 'career_avg_batsman'

In [158]:
def career_avg_batsman(p_id):
    career_avg_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[2]
    career_avg_batsman=career_avg_batsman[["Span","Inns","Runs","HS","Ave","SR","100","50","0","4s","6s"]]

    career_avg_batsman.insert(0, "P_id", [p_id], True)

    filepath = Path('Csv/Career_avg_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    career_avg_batsman.to_csv(filepath,  mode='a', index=False, header=hdr)  
    career_avg_batsman


Table: 'vs_country_batsman'

In [159]:
def vs_country_batsman(p_id):
    vs_country_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[3]
    vs_country_batsman=vs_country_batsman[["Grouping","Inns","Runs","HS","Ave","SR"]]

    vs_country_batsman.rename(columns = {'Grouping':'Country'}, inplace = True)
    vs_country_batsman.insert(0, "P_id", p_id, True)

    countries=['v Pakistan', 'v Australia', 'v England', 'v New Zealand', 'v West Indies', 'v Sri Lanka', 'v Bangladesh', 'v South Africa']

    select_vs_country_batsman = vs_country_batsman.loc[ vs_country_batsman['Country'].isin(countries)  ] 

    filepath = Path('Csv/Vs_country_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    select_vs_country_batsman.to_csv(filepath, mode='a', index=False, header=hdr)  
    select_vs_country_batsman

Table: 'home_vs_away_batsman'

In [160]:
def home_vs_away_batsman(p_id):
    home_vs_away_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[3]
    home_vs_away_batsman=home_vs_away_batsman[["Grouping","Inns","Runs","HS","Ave","SR"]]

    home_vs_away_batsman.rename(columns = {'Grouping':'Venue'}, inplace = True)
    home_vs_away_batsman.insert(0, "P_id", p_id, True)

    venues=['home', 'away', 'neutral']

    select_home_vs_away_batsman = home_vs_away_batsman.loc[ home_vs_away_batsman['Venue'].isin(venues)  ] 

    filepath = Path('Csv/Home_vs_away_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    select_home_vs_away_batsman.to_csv(filepath, mode='a', index=False, header=hdr)  
    select_home_vs_away_batsman

Table: 'yearly_stats_batsman'

In [161]:
def yearly_stats_batsman(p_id):
    yearly_stats_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[3]
    yearly_stats_batsman=yearly_stats_batsman[["Grouping","Inns","Runs","HS","Ave","SR"]]

    yearly_stats_batsman.rename(columns = {'Grouping':'Year'}, inplace = True)
    yearly_stats_batsman.insert(0, "P_id", p_id, True)

    years=['home', 'away', 'neutral']

    select_yearly_stats_batsman = yearly_stats_batsman.loc[ yearly_stats_batsman['Year'].str.startswith('year ', na=False)  ] 

    filepath = Path('Csv/Yearly_stats_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    select_yearly_stats_batsman.to_csv(filepath, mode='a', index=False, header=hdr)  
    select_yearly_stats_batsman

#Main function below:

In [162]:
#253802: Virat Kohli (BAT)
#34102: Rohit Gurunath Sharma (BAT)
#422108: Kannaur Lokesh Rahul (BAT)
#234675: Ravindrasinh Anirudhsinh Jadeja (ALL)
#625371: Hardik Himanshu Pandya (ALL)

player_ids=['253802', '34102', '422108', '234675', '625371']

os.system("rm -f Csv/*_batsman.csv")

for pid in player_ids:
    career_avg_batsman(pid)
    vs_country_batsman(pid)
    home_vs_away_batsman(pid)
    yearly_stats_batsman(pid)
    